# Gemma in the Hugging Face Ecosystem

<p align="center">
  <img src="https://github.com/sanchit-gandhi/notebooks/blob/main/gemma_pipeline.jpg?raw=true" width="800"/>
</p>

## Inference with Transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, do_sample=True, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

## Datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco")

In [ ]:
sample = dataset["train"][0]
sample

## Training with TRL

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

Define training arguments:

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_steps=2,
    gradient_checkpointing=True,
    output_dir="gemma-2b-fine-tuned",
)

Instantiate supervised fine-tuning (SFT) trainer:

In [ ]:
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=1024,
)

In [ ]:
trainer.train()